# Testing different storage methods 
---
See if there's a better way to read/write the data in different formats.
Want to 
1) speed up read/write time 
2) find a way to store metadata in the file without having to save as columns
3) save disk space (compression?)


Try/investigate:
- pytables
- sqlite
- parquet
- apache arrow

In [ ]:
from spaxelsleuth import load_user_config
try:
    load_user_config("/Users/u5708159/Desktop/spaxelsleuth_test/.myconfig.json")
except FileNotFoundError:
    load_user_config("/home/u5708159/.spaxelsleuthconfig.json")
from spaxelsleuth.loaddata.sami import load_sami_df

In [ ]:
DEBUG = False

In [ ]:
# Load the DataFrame
df = load_sami_df(ncomponents="recom",
                bin_type="default",
                eline_SNR_min=5,
                eline_ANR_min=3,
                correct_extinction=True,
                debug=False)
